In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [2]:
dict_tablas = pickle.load(open('dict_tablas','rb'))
df_historical_data = pd.read_csv(r'Datasets\Clean_Fifa_WorldCup_Matches.csv')
df_fixture = pd.read_csv(r'Datasets\Fifa_WorldCup_Fixture.csv')

In [3]:
dict_tablas['Grupo F']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Belgium,0,0,0,0,0,0,0,0
1,2,Canada,0,0,0,0,0,0,0,0
2,3,Morocco,0,0,0,0,0,0,0,0
3,4,Croatia,0,0,0,0,0,0,0,0


## 1.- Calcular Nivel de equipo

In [4]:
#Dividir df en df_home y df_away
df_historical_data


,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3
...,...,...,...,...,...
895,Russia,Croatia,2018,2,2
896,France,Belgium,2018,1,0
897,Croatia,England,2018,2,1
898,Belgium,England,2018,2,0


In [5]:
#Dividir df en df_home y df_away
df_home = df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away = df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

In [6]:
#Renombrar Columnas
df_home = df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalScored','AwayGoals':'GoalConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalConceded','AwayGoals':'GoalScored'})

In [7]:
#Concatenar df_home y df_away, hacer grupo por team y calcular promedio
df_team_strenght = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()
df_team_strenght

,GoalScored,GoalConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


In [8]:
def prediction_points(home,away):
    if home in df_team_strenght.index and away in df_team_strenght.index:
        #Para obtener lambda vamos a multiplicar el promedio de goles anotas del
        #equipo rival por el promedio de goles recibidos del equipo visitante
        #goals_scored * goals_conceded
        lamb_home = df_team_strenght.at[home, 'GoalScored'] * df_team_strenght.at[away,'GoalConceded']
        lamb_away = df_team_strenght.at[away, 'GoalScored'] * df_team_strenght.at[home,'GoalConceded']
        prob_home, prob_away, prob_draw = 0,0,0
        for x in range(0,11): #Number of goals home team
            for y in range(0,11): #Number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0,0)

## 2.1- Probamos nuestra función

In [9]:
#Testear con partidos: Argentina - México, England - United States, Qatar(H) -Ecuador
prediction_points('Argentina','Mexico')
prediction_points('England','United States')
#prediction_points('Qatar','Ecuador')

(2.2356147635326007, 0.5922397535606193)

## 3.- Prediciendo Resultados de Mundial

In [10]:
#Dividiendo fixture en grupo, octavos, cuartos,...
df_fixture_gruop_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semis = df_fixture[60:62].copy()
df_fixture_third_place = df_fixture[62:63].copy()
df_fixture_final = df_fixture[63:].copy()
df_fixture_gruop_48

,Home,Score,Away,Year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
5,Netherlands,Match 36,Qatar,2022
6,England,Match 3,Iran,2022
7,United States,Match 4,Wales,2022
8,Wales,Match 17,Iran,2022
9,England,Match 20,United States,2022


In [33]:
#Correr todos los partidos de la fase de grupos y actualizamos las tablas de cada grupo
for group in dict_tablas:
    print(f"Procesando grupo: {group}")
    teams_in_group = dict_tablas[group]['Team'].values
    print(f"Equipos en el grupo: {teams_in_group}")
    
    # Verificar contenido de teams_in_group
    print("Contenido de teams_in_group:")
    for team in teams_in_group:
        print(f"'{team}'")
    
    teams_in_group = [team.strip().lower() for team in dict_tablas[group]['Team'].values]
    df_fixture_gruop_48['Home'] = df_fixture_gruop_48['Home'].str.strip().str.lower()
    df_fixture_gruop_48['Away'] = df_fixture_gruop_48['Away'].str.strip().str.lower()

    df_fixture_gruop_6 = df_fixture_gruop_48[df_fixture_gruop_48['Home'].isin(teams_in_group)]
    print(f"Fixture filtrado para el grupo: {df_fixture_gruop_6}")
    
    if df_fixture_gruop_6.empty:
        print(f"No hay partidos para los equipos en el grupo: {group}")
        continue
    
    for index, row in df_fixture_gruop_6.iterrows():
        home, away = row['Home'], row['Away']
        print(f"Partido: {home} vs {away}")
        
        points_home, points_away = prediction_points(home, away)
        print(f"Puntos predichos - Home: {points_home}, Away: {points_away}")
        
        dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home
        dict_tablas[group].loc[dict_tablas[group]['Team'] == away, 'Pts'] += points_away
    
    dict_tablas[group] = dict_tablas[group].sort_values('Pts', ascending=False).reset_index(drop=True)
    dict_tablas[group] = dict_tablas[group][['Team','Pts']]
    dict_tablas[group] = dict_tablas[group].round(0)
    print(f"Tabla actualizada para el grupo {group}:")
    print(dict_tablas[group])

Procesando grupo: Grupo A
Equipos en el grupo: ['Qatar (H)' 'Ecuador' 'Senegal' 'Netherlands']
Contenido de teams_in_group:
'Qatar (H)'
'Ecuador'
'Senegal'
'Netherlands'
Fixture filtrado para el grupo:           Home     Score         Away  Year
1      senegal   Match 2  netherlands  2022
3  netherlands  Match 19      ecuador  2022
4      ecuador  Match 35      senegal  2022
5  netherlands  Match 36        qatar  2022
Partido: senegal vs netherlands
Puntos predichos - Home: 0, Away: 0
Partido: netherlands vs ecuador
Puntos predichos - Home: 0, Away: 0
Partido: ecuador vs senegal
Puntos predichos - Home: 0, Away: 0
Partido: netherlands vs qatar
Puntos predichos - Home: 0, Away: 0
Tabla actualizada para el grupo Grupo A:
          Team  Pts
0    Qatar (H)    0
1      Ecuador    0
2      Senegal    0
3  Netherlands    0
Procesando grupo: Grupo B
Equipos en el grupo: ['England' 'Iran' 'United States' 'Wales']
Contenido de teams_in_group:
'England'
'Iran'
'United States'
'Wales'
Fixture fil

In [35]:
#mostrar tabla actualizada
dict_tablas['Grupo A']

,Team,Pts
0,Qatar (H),0
1,Ecuador,0
2,Senegal,0
3,Netherlands,0
